In [1]:
cd ".."

c:\Users\offic\Desktop\Big Data Analysis and Transaction Predictive Modelling - ARC Transaction Dataset


In [2]:
import json
import pandas as pd
import re
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [8]:
with open("Configurations.yaml","r") as configurations:
    _Configurations = json.load(configurations)

## Img V

In [3]:
def convertStringToDateTime(string):
    match = re.search("\d\d\d\d-\d\d-\d\d",string)
    return datetime.strptime(match.group(),'%Y-%m-%d') 
    #(x-y).days
    
def DateDifference(stringDate1, stringDate2):
    date1 = convertStringToDateTime(stringDate1)
    date2 = convertStringToDateTime(stringDate2)
    return abs((date1-date2).days)

In [4]:
def OneWayTripMergeSegmentHelper(transactionKey, frame):
    _dfDict = {"TRANSACTION_KEY":[], "TRIP_TYPE":[], "NUMBER_OF_SEGMENTS":[], "ORIGIN_AIRPORT":[], "DESTINATION_AIRPORT":[], "DEPARTURE_DATE":[]}
   
    _dfDict["TRANSACTION_KEY"].append(transactionKey)
    _dfDict["TRIP_TYPE"] = "'OW'"
    frame = frame.sort_values(by="SEG_NUMBER")

        
    firstRow = frame.iloc[0]
    lastRow = frame.iloc[-1]
    _dfDict["NUMBER_OF_SEGMENTS"] = lastRow["SEG_NUMBER"]
    _dfDict["ORIGIN_AIRPORT"] = firstRow["ORIGIN_AIRPORT"]
    _dfDict["DESTINATION_AIRPORT"] = lastRow["DESTINATION_AIRPORT"]
    _dfDict["DEPARTURE_DATE"] = firstRow["DEPARTURE_DATE"]

    return pd.DataFrame(_dfDict)

In [5]:
def ReturnTripMergeSegmentHelper(transactionKey, frame, originDepartureDate = None):
    _dfDict = {"TRANSACTION_KEY":[], "TRIP_TYPE":[], "NUMBER_OF_SEGMENTS":[], "ORIGIN_AIRPORT":[], "DESTINATION_AIRPORT":[], "DEPARTURE_DATE":[]}

    if frame.shape[0] == 0:
        return pd.DataFrame(_dfDict) # If the frame is empty

    frame.reset_index(drop=True)
    frame = frame.sort_values(by="SEG_NUMBER")
    firstRow = frame.iloc[0]

    originSegmentNumber = firstRow["SEG_NUMBER"]
    lastSegmentNumber = originSegmentNumber

    originAirport = firstRow["ORIGIN_AIRPORT"]

    destinationAirport = firstRow["DESTINATION_AIRPORT"]

    if originDepartureDate is None:
        originDepartureDate = firstRow["DEPARTURE_DATE"]
    previousDate = originDepartureDate
    
    visitedAirport = set([])
    visitedAirport.add(originAirport)
    visitedAirport.add(destinationAirport)

    locationalIndex = 1
    
    for index, row in frame[1:].iterrows():

        if row["ORIGIN_AIRPORT"] != destinationAirport:
            _dfDict["TRANSACTION_KEY"].append(transactionKey)
            _dfDict["TRIP_TYPE"].append("'OW'")
            _dfDict["NUMBER_OF_SEGMENTS"].append(lastSegmentNumber - originSegmentNumber + np.int64(1))
            _dfDict["ORIGIN_AIRPORT"].append(originAirport)
            _dfDict["DESTINATION_AIRPORT"].append(destinationAirport)
            _dfDict["DEPARTURE_DATE"].append(originDepartureDate)
            return pd.concat([pd.DataFrame(_dfDict),ReturnTripMergeSegmentHelper(transactionKey,frame[locationalIndex:])],axis=0) # Leveraging Recursion

        else:
            try: # Valid-Departure Date
                currentDate = row["DEPARTURE_DATE"]

                if DateDifference(currentDate,previousDate)<2 and row["DESTINATION_AIRPORT"] not in visitedAirport:
                    lastSegmentNumber = row["SEG_NUMBER"]
                    destinationAirport = row["DESTINATION_AIRPORT"]
                    previousDate = row["DEPARTURE_DATE"]
                    visitedAirport.add(destinationAirport)
                    locationalIndex+=1

                else:
                    _dfDict["TRANSACTION_KEY"].append(transactionKey)
                    _dfDict["TRIP_TYPE"].append("'OW'")
                    _dfDict["NUMBER_OF_SEGMENTS"].append(lastSegmentNumber - originSegmentNumber + np.int64(1))
                    _dfDict["ORIGIN_AIRPORT"].append(originAirport)
                    _dfDict["DESTINATION_AIRPORT"].append(destinationAirport)
                    _dfDict["DEPARTURE_DATE"].append(originDepartureDate)
                    return pd.concat([pd.DataFrame(_dfDict),ReturnTripMergeSegmentHelper(transactionKey,frame[locationalIndex:])],axis=0) # Leveraging Recursion

            except: # Non Flight Segment
                if row["DESTINATION_AIRPORT"] not in visitedAirport:
                    lastSegmentNumber = row["SEG_NUMBER"]
                    destinationAirport = row["DESTINATION_AIRPORT"]
                    visitedAirport.add(destinationAirport)
                    locationalIndex+=1

                else:
                    _dfDict["TRANSACTION_KEY"].append(transactionKey)
                    _dfDict["TRIP_TYPE"].append("'OW'")
                    _dfDict["NUMBER_OF_SEGMENTS"].append(lastSegmentNumber - originSegmentNumber + np.int64(1))
                    _dfDict["ORIGIN_AIRPORT"].append(originAirport)
                    _dfDict["DESTINATION_AIRPORT"].append(destinationAirport)
                    _dfDict["DEPARTURE_DATE"].append(originDepartureDate)
                    return pd.concat([pd.DataFrame(_dfDict),ReturnTripMergeSegmentHelper(transactionKey,frame[locationalIndex:], previousDate)],axis=0) # Leveraging Recursion

    _dfDict["TRANSACTION_KEY"].append(transactionKey)
    _dfDict["TRIP_TYPE"].append("'OW'")
    _dfDict["NUMBER_OF_SEGMENTS"].append(lastSegmentNumber - originSegmentNumber + np.int64(1))
    _dfDict["ORIGIN_AIRPORT"].append(originAirport)
    _dfDict["DESTINATION_AIRPORT"].append(destinationAirport)
    _dfDict["DEPARTURE_DATE"].append(originDepartureDate)
    return pd.DataFrame(_dfDict)
                

In [6]:
def MergeSegment(df):
    dataFrame = pd.DataFrame({"TRANSACTION_KEY":[], "TRIP_TYPE":[], "NUMBER_OF_SEGMENTS":[], "ORIGIN_AIRPORT":[], "DESTINATION_AIRPORT":[], "DEPARTURE_DATE":[]}) # The New Dataframe
    group = df.groupby(by=["TRANSACTION_KEY","TRIP_TYPE"])
    noOfGroups = len(group)
    print("Number of Groups : " , noOfGroups)
    currentNumber = 1
    for node, frame in tqdm(group):
        tripType = node[1]
        if tripType == "'OW'": # Processing One-Way Trip
            dataFrame = pd.concat([dataFrame,OneWayTripMergeSegmentHelper(node[0], frame)],axis=0)
        elif tripType == "'RT'": # Processing One Way Trip
            dataFrame = pd.concat([dataFrame,ReturnTripMergeSegmentHelper(node[0],frame)],axis=0)
        else: # Processing Complex Trip
            dataFrame = pd.concat([dataFrame,ReturnTripMergeSegmentHelper(node[0],frame)],axis=0)
        currentNumber+=1

    return dataFrame.reset_index(drop=True)

In [27]:
df = pd.read_csv(_Configurations["Dataset"]["Processed"]["SampleDataset"])
invalidTransactions = df["TRANSACTION_KEY"][(df["DEPARTURE_DATE"]=="'\\N'") & (df["MARKETING_AIRLINE_CD"]!="'V'")].unique()
df = df[~df["TRANSACTION_KEY"].isin(invalidTransactions)]

In [28]:
lst = ["'T-1808639477801388902'", "'T-1808639477801466628'", "'T-1808639477801808735'", "'T-1833843851600035141'", "'T-1833843851600065553'", "'T-1808639478100000606'","'T-1833843851800015549'","'T-1809239577600177000'","'T-1801800000000318159'"]
df = df[["TRANSACTION_KEY", "TRANSACTION_TYPE", "TRIP_TYPE","SEG_NUMBER","ORIGIN_AIRPORT","DESTINATION_AIRPORT","DEPARTURE_DATE"]][df["TRANSACTION_KEY"].isin(lst)]

In [29]:
result = MergeSegment(df)
result

Number of Groups :  9


  0%|          | 0/9 [00:00<?, ?it/s]

,TRANSACTION_KEY,TRIP_TYPE,NUMBER_OF_SEGMENTS,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DATE
0,'T-1801800000000318159','OW',3.0,'AUS','MAD','2018-07-14'
1,'T-1801800000000318159','OW',2.0,'MAD','AUS','2018-08-02'
2,'T-1808639477801388902','OW',1.0,'NNG','KHN','2018-04-08'
3,'T-1808639477801388902','OW',1.0,'KHN','NNG','2018-04-14'
4,'T-1808639477801466628','OW',2.0,'YWG','YSB','2018-03-29'
5,'T-1808639477801466628','OW',2.0,'YSB','YWG','2018-04-02'
6,'T-1808639477801808735','OW',3.0,'YUL','CMB','2018-07-11'
7,'T-1808639477801808735','OW',3.0,'CMB','YUL','2018-07-30'
8,'T-1808639478100000606','OW',1.0,'BKK','HEL','2018-03-22'
9,'T-1809239577600177000','OW',1.0,'CDG','HKG','2018-03-22'


In [30]:
result = result.drop(['TRIP_TYPE'],axis=1)

In [31]:
relationalDf = df[["TRANSACTION_KEY","TRIP_TYPE"]].drop_duplicates(keep="first")
relationalDf

,TRANSACTION_KEY,TRIP_TYPE
10,'T-1808639477801388902','RT'
222,'T-1808639477801466628','RT'
1160,'T-1808639477801808735','RT'
2045,'T-1833843851600035141','RT'
2192,'T-1833843851600065553','RT'
2283,'T-1833843851800015549','XX'
2561,'T-1809239577600177000','XX'
3580,'T-1808639478100000606','RT'
7188479,'T-1801800000000318159','XX'


In [32]:
result.merge(relationalDf,on="TRANSACTION_KEY",how="left")

,TRANSACTION_KEY,NUMBER_OF_SEGMENTS,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DATE,TRIP_TYPE
0,'T-1801800000000318159',3.0,'AUS','MAD','2018-07-14','XX'
1,'T-1801800000000318159',2.0,'MAD','AUS','2018-08-02','XX'
2,'T-1808639477801388902',1.0,'NNG','KHN','2018-04-08','RT'
3,'T-1808639477801388902',1.0,'KHN','NNG','2018-04-14','RT'
4,'T-1808639477801466628',2.0,'YWG','YSB','2018-03-29','RT'
5,'T-1808639477801466628',2.0,'YSB','YWG','2018-04-02','RT'
6,'T-1808639477801808735',3.0,'YUL','CMB','2018-07-11','RT'
7,'T-1808639477801808735',3.0,'CMB','YUL','2018-07-30','RT'
8,'T-1808639478100000606',1.0,'BKK','HEL','2018-03-22','RT'
9,'T-1809239577600177000',1.0,'CDG','HKG','2018-03-22','XX'
